In [1]:
# from Recommenders.Hybrid.SimilarityMergedHybridRecommender import SimilarityMergedHybridRecommender
# from src.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaCBFRecommender import RP3betaCBFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from Recommenders.Hybrid.GeneralizedMergedHybridRecommender import (
    GeneralizedMergedHybridRecommender,
)
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_ElasticNet
from Recommenders.Implicit.FeatureCombinedImplicitALSRecommender import (
    FeatureCombinedImplicitALSRecommender,
)
from Recommenders.Hybrid.GeneralizedSimilarityMergedHybridRecommender import (
    GeneralizedSimilarityMergedHybridRecommender,
)
from Utils.ICM_preprocessing import *
from Utils.data_loader import load_URM, load_ICM
from Utils.confidence_scaling import *
from Utils.write_submission import write_submission
from Data_manager.split_functions.split_train_validation_random_holdout import (
    split_train_in_two_percentage_global_sample,
)
from Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from bayes_opt import BayesianOptimization

%env OPENBLAS_NUM_THREADS=1


env: OPENBLAS_NUM_THREADS=1


In [2]:
URM_all = load_URM("input/data_train.csv")
ICM_all = load_ICM("input/data_ICM_channel.csv")

# ICM_combined = combine(ICM=ICM_all, URM = URM_all)


In [3]:
URMs_train = []
URMs_validation = []

for k in range(5):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(
        URM_all, train_percentage=0.80
    )
    URMs_train.append(URM_train)
    URMs_validation.append(URM_validation)


In [4]:
evaluator_validation = K_Fold_Evaluator_MAP(
    URMs_validation, cutoff_list=[10], verbose=False
)

ICMs_combined = []
for URM in URMs_train:
    ICMs_combined.append(combine(ICM=ICM_all, URM=URM))


In [5]:
IALS_recommenders = []
rp3betaCBF_recommenders = []
#userRP3beta_recommenders = []
itemKNN_recommenders = []

for index in range(len(URMs_train)):
    IALS_recommenders.append(
        FeatureCombinedImplicitALSRecommender(
            URM_train=URMs_train[index], ICM_train=ICM_all, verbose=True
        )
    )

    # TODO: train this again by itself
    IALS_recommenders[index].fit(
        factors=int(398.601583855084),
        regularization=0.01,
        use_gpu=False,
        iterations=int(94.22855449116447),
        num_threads=6,
        confidence_scaling=linear_scaling_confidence,
        **{
            "URM": {"alpha": 42.07374324671451}, 
            "ICM": {"alpha": 41.72067133975204}}
    )


    rp3betaCBF_recommenders.append(
        RP3betaCBFRecommender(
            URM_train=URMs_train[index], ICM_train=ICMs_combined[index], verbose=False
        )
    )

    # TODO: train this again by itself
    rp3betaCBF_recommenders[index].fit(
        topK=int(529.1628484087545),
        alpha=0.45304737831676245,
        beta=0.226647894170121,
        implicit=True,
    )


    # userRP3beta_recommenders.append(
    #     UserRP3betaRecommender(
    #         URM_train=ICMs_combined[index].T,
    #         verbose=False
    #     )
    # )

    # userRP3beta_recommenders[index].fit(
    #     topK=int(252.2),
    #     alpha=0.2658,
    #     beta=0.2847,
    #     implicit=False
    # )



    # itemKNN_recommenders.append(
    #     ItemKNNCFRecommender(
    #         URM_train=URMs_train[index],
    #         verbose=False
    #     )
    # )
    
    # itemKNN_recommenders[index ].fit(
    #     topK=700,
    #     shrink=0
    # )



100%|██████████| 94/94 [15:28<00:00,  9.87s/it]


FeatureCombinedImplicitALSRecommender: URM Detected 2 ( 0.0%) items with no interactions.


 19%|█▉        | 18/94 [03:33<15:02, 11.87s/it]


KeyboardInterrupt: 

## Tuning hybrid weights

In [ ]:
tuning_params = {
    "hybridWeight1":  (0, 1),
    "hybridWeight2":  (0, 1)
    #"hybridWeight3": (0, 1)
}

results = []

def BO_func(
    hybridWeight1,
    hybridWeight2
    #hybridWeight3
):
    recommenders = []

    for index in range(len(URMs_train)):
        recommender = GeneralizedMergedHybridRecommender(
            URM_train=URMs_train[index],
            recommenders=[
                IALS_recommenders[index],
                rp3betaCBF_recommenders[index],
                # userRP3beta_recommenders[index],
                #itemKNN_recommenders[index]
                
            ],
            verbose=True,
        )

        totalWeight = hybridWeight1 + hybridWeight2  # +hybridWeight3
        recommender.fit(
            alphas=[
                hybridWeight1 / totalWeight,
                hybridWeight2 / totalWeight
                #hybridWeight3 / totalWeight
            ]
        )

        recommenders.append(recommender)

    result = evaluator_validation.evaluateRecommender(recommenders)
    results.append(result)

    return sum(result) / len(result)

In [ ]:
optimizer = BayesianOptimization(
    f=BO_func,
    pbounds=tuning_params,
    verbose=5,
    random_state=5,
)

optimizer.maximize(
    init_points=30,
    n_iter=50,
)

|   iter    |  target   | hybrid... | hybrid... | hybrid... |
-------------------------------------------------------------
|  1        |  0.1933   |  0.222    |  0.8707   |  0.2067   |
|  2        |  0.1996   |  0.9186   |  0.4884   |  0.6117   |
|  3        |  0.1962   |  0.7659   |  0.5184   |  0.2968   |
|  4        |  0.2057   |  0.1877   |  0.08074  |  0.7384   |
|  5        |  0.205    |  0.4413   |  0.1583   |  0.8799   |
|  6        |  0.1968   |  0.2741   |  0.4142   |  0.2961   |
|  7        |  0.1986   |  0.6288   |  0.5798   |  0.5999   |
|  8        |  0.1978   |  0.2658   |  0.2847   |  0.2536   |
|  9        |  0.1993   |  0.3276   |  0.1442   |  0.1656   |
|  10       |  0.1932   |  0.9639   |  0.9602   |  0.1884   |
|  11       |  0.2033   |  0.02431  |  0.2046   |  0.6998   |
|  12       |  0.2074   |  0.7795   |  0.02293  |  0.5777   |
|  13       |  0.1991   |  0.001642 |  0.5155   |  0.6398   |
|  14       |  0.2037   |  0.9856   |  0.2591   |  0.8025   |
|  15   

/var/folders/jh/1p6pr_n9211b_s8qr8gkyx600000gn/T/ipykernel_20542/4010531221.py:32: RuntimeWarning: invalid value encountered in double_scalars
  hybridWeight1 / totalWeight,
/var/folders/jh/1p6pr_n9211b_s8qr8gkyx600000gn/T/ipykernel_20542/4010531221.py:33: RuntimeWarning: invalid value encountered in double_scalars
  hybridWeight2 / totalWeight,
/var/folders/jh/1p6pr_n9211b_s8qr8gkyx600000gn/T/ipykernel_20542/4010531221.py:34: RuntimeWarning: divide by zero encountered in double_scalars
  hybridWeight3 / totalWeight
/Users/massimo/Development/GitHub/recsys-2021/Recommenders/Hybrid/GeneralizedMergedHybridRecommender.py:38: RuntimeWarning: invalid value encountered in multiply
  result = result + self.alphas[index]*self.recommenders[index]._compute_item_score(user_id_array,items_to_compute)


|  33       |  0.0      |  0.0      |  0.0      |  1.0      |
|  34       |  0.1975   |  0.08348  |  0.3615   |  0.3084   |
|  35       |  0.2014   |  0.1786   |  0.4914   |  0.9944   |
|  36       |  0.1976   |  1.0      |  1.0      |  0.8387   |
|  37       |  0.2072   |  0.8282   |  0.02142  |  0.9766   |
|  38       |  0.1981   |  0.0      |  1.0      |  1.0      |
|  39       |  0.2035   |  0.3616   |  0.2395   |  0.8083   |
|  40       |  0.1949   |  0.7023   |  0.8651   |  0.3669   |


/var/folders/jh/1p6pr_n9211b_s8qr8gkyx600000gn/T/ipykernel_20542/4010531221.py:32: RuntimeWarning: invalid value encountered in double_scalars
  hybridWeight1 / totalWeight,
/var/folders/jh/1p6pr_n9211b_s8qr8gkyx600000gn/T/ipykernel_20542/4010531221.py:33: RuntimeWarning: invalid value encountered in double_scalars
  hybridWeight2 / totalWeight,
/var/folders/jh/1p6pr_n9211b_s8qr8gkyx600000gn/T/ipykernel_20542/4010531221.py:34: RuntimeWarning: invalid value encountered in double_scalars
  hybridWeight3 / totalWeight


|  41       |  0.0      |  0.0      |  0.0      |  0.0      |
|  42       |  0.1916   |  0.5487   |  0.3777   |  0.0      |
|  43       |  0.1909   |  0.0      |  1.0      |  0.0      |
|  44       |  0.1966   |  0.2103   |  1.0      |  0.7108   |
|  45       |  0.1981   |  0.4093   |  0.6674   |  0.6441   |
|  46       |  0.2086   |  0.9974   |  0.0032   |  0.44     |
|  47       |  0.0955   |  1.0      |  0.0      |  0.0      |
|  48       |  0.1998   |  0.6228   |  0.2424   |  0.3031   |
|  49       |  0.2029   |  0.6638   |  0.3712   |  1.0      |
|  50       |  0.1977   |  0.2576   |  0.2815   |  0.2405   |
|  51       |  0.2005   |  1.0      |  0.6406   |  1.0      |
|  52       |  0.1909   |  0.0      |  0.6371   |  0.0      |
|  53       |  0.1912   |  0.4882   |  1.0      |  0.0      |
|  54       |  0.2003   |  1.0      |  0.2893   |  0.3817   |
|  55       |  0.1968   |  0.9928   |  0.7919   |  0.5369   |
|  56       |  0.2009   |  0.0525   |  0.3481   |  0.6431   |
|  57   

In [ ]:
best = optimizer.max
print("TARGET MAP", best["target"])

hw1 = best["params"]["hybridWeight1"]
hw2 = best["params"]["hybridWeight2"]
#hw3 = best["params"]["hybridWeight3"]

print("HYBRID WEIGHTS", hw1, hw2, hw3)

totalWeight = hw1 + hw2 #+ hw3
totalWeight

alpha1, alpha2 = hw1 / totalWeight, hw2 / totalWeight#, hw3 / totalWeight

print("ALPHAS", alpha1, alpha2)

TARGET MAP 0.20864964141248468
HYBRID WEIGHTS 0.9974356760093824 0.0031999638023508803 0.4400192671890031
ALPHAS 0.6923487860711341 0.0022211868968765084 0.3054300270319894


## Train final model with tuned hyperparameters

In [ ]:
recommender = GeneralizedMergedHybridRecommender(
    URM_train=URMs_train[0], # maybe we can take a better option instead of taking the first one of everything?
    recommenders=[
        IALS_recommenders[0],
        rp3betaCBF_recommenders[0],
        #itemKNN_recommenders[0]
        # userRP3beta_recommenders[index],
    ],
    verbose=True,
)

recommender.fit(
    alphas=[
        alpha1,
        alpha2
        #alpha3
    ]
)


In [ ]:
write_submission(recommender=recommender, target_users_path="input/data_target_users_test.csv",
                    out_path='output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))

100%|██████████| 13650/13650 [01:03<00:00, 213.95it/s]


In [ ]:
print('output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))

output/FeatureCombinedImplicitALSRP3betaCBFItemKNNCFHybridRecommender_submission.csv


In [ ]:
# p3alpha_recommender = P3alphaRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
# p3alpha_recommender.fit(
#     topK=int(212.8832860130684),
#     alpha=0.4729294763382114,
#     implicit=True
# )


In [ ]:
# RP3 BETA (Combined) -----------------------------------------

# rp3betaCombined_recommender= RP3betaCBFRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_combined,
#     verbose=False
# )

# rp3betaCombined_recommender.fit(
#     topK=int(529.1628484087545),
#     alpha=0.45304737831676245,
#     beta=0.226647894170121,
#     implicit=True
# )


In [ ]:
# IALS -----------------------------------------

# IALS_recommender= FeatureCombinedImplicitALSRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_all,
#     verbose=True
# )

# IALS_recommender.fit(
#     factors=int(398.601583855084),
#     regularization=0.01,
#     use_gpu=False,
#     iterations=int(94.22855449116447),
#     num_threads=6,
#     confidence_scaling=linear_scaling_confidence,
#     **{
#         'URM': {"alpha": 42.07374324671451},
#         'ICM': {"alpha": 41.72067133975204}
#     }
# )


In [ ]:
# SLIM -----------------------------------------

# SLIM_recommender = MultiThreadSLIM_ElasticNet(
#         URM_train=ICM_combined.T,
#         verbose=False
#     )

# SLIM_recommender.fit(
#     alpha=0.00026894910579512645,
#     l1_ratio=0.08074126876487486,
#     topK=int(395.376118479588),
#     workers=6
# )

# SLIM_recommender.URM_train = URM_all


In [ ]:
# OTHER STUFF ----------------------------------

# rp3betaCBF_recommender= RP3betaCBFRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_all,
#     verbose=False
# )
#
# rp3betaCBF_recommender.fit(
#     topK=int(117.1),
#     alpha=0.9882,
#     beta=0.7703,
#     implicit=False
# )

# itemKNN_recommender= ItemKNNCFRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
# itemKNN_recommender.fit(
#     topK=100,
#     shrink=50
# )
#
# pureSVD_recommender= PureSVDItemRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
#
# pureSVD_recommender.fit(
#     num_factors=772,
#     topK= 599
# )


In [ ]:
# Generalized Merged Hybrid -----------------------------------------

# recommender = GeneralizedMergedHybridRecommender(
#     URM_train=URM_all,
#     recommenders=[
#         IALS_recommender,
#         rp3betaCombined_recommender
#         #SLIM_recommender
#     ],
#     verbose=False
# )


In [ ]:
# recommender.fit(
#     alphas=[
#         0.6836750866517823,
#         0.45969038157844144,
#         #0.2723405593515382
#     ]
# )


In [ ]:
# write_submission(recommender=recommender, target_users_path="input/data_target_users_test.csv",
#                     out_path='output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))
